In [1]:
import glob
from deliverable.code.tsp.TSP_Formulation_Methods import *
import matplotlib.pyplot as plt
import numpy as np
import os

distances_original_matrix = np.loadtxt("./data/matriz-rutas-granada")

# First, we load a set of computed lambdas (all if not specified) and do some statistical analysis on them
# The lambdas computed are all from staring factor 0.01 and 5 iterations (see QUBO_optimized_lambdas.ipynb)

In [ ]:
concrete_simulations = "lambdas"
files_to_find = "./data/lamdasOptimized/"+concrete_simulations+"*"
files = glob.glob(files_to_find)

all_weights = []

for file in files:
    weights = np.loadtxt(file)
    all_weights.append(weights)

all_weights = np.array(all_weights)  # Shape: (num_files, 5)

means = np.mean(all_weights, axis=0)
variances = np.var(all_weights, axis=0)

for i in range(5):
    print(f"Weight λ{i+1}: Mean = {means[i]:.4f}, Variance = {variances[i]:.4f}")

plt.figure(figsize=(10, 5))
for i in range(5):
    plt.hist(all_weights[:, i], bins=10, alpha=0.6, label=f"λ{i+1}")
plt.xlabel("Valor del peso")
plt.ylabel("Frecuencia")
plt.title("Distribución de los pesos óptimos")
plt.legend()
plt.show()

In [ ]:
plt.boxplot(all_weights, labels=[f"λ{i+1}" for i in range(5)])
plt.ylabel("Weight value")
plt.title("Variability of optimal weights")
plt.show()

# Now we try to solve using the mean lambdas. We check the accuracy for each kind of combination

In [ ]:
N = 5
p = 2
num_reads_solver = 400
distances_N_stops_normalized = distances_original_matrix[:N,:N]/np.max(distances_original_matrix[:N,:N])

script_dir = os.getcwd()
concrete_simulations = "lambdas"
files_to_find = os.path.join(script_dir, f"data/lamdasOptimized/{concrete_simulations}*")
list_of_files = glob.glob(files_to_find)

mean_lambdas = load_lambda_means(list_of_files)

print("Mean lambdas: ", mean_lambdas)

all_start_end_combinations = generate_all_start_end_combinations(N, L=1)

solutions_analysis = np.zeros((len(all_start_end_combinations), 3))

for index, (startNodes, endNodes) in enumerate(all_start_end_combinations):
    startNode = startNodes[0]
    endNode = endNodes[0]
    solutions_analysis[index][0] = startNode
    solutions_analysis[index][1] = endNode
    Q_matrix,_ = create_QUBO_matrix(distances_N_stops_normalized, p, startNode, endNode, mean_lambdas)
    solution_array, _ = solve_qubo_with_Dwave(Q_matrix, num_reads_solver)
    solutions_analysis[index][2] = int(check_solution_return(solution_array, N, p, startNode, endNode))

show_statistics_lambdas(solutions_analysis, N)

check_solution(solution_array, N, p, N-2, N-1)
draw_solution_graph(solution_array, distances_N_stops_normalized, p, solutions_analysis[-1][0], solutions_analysis[-1][1])


# Now we visualize the value for each optimized penalty weight individualized for N and p

In [ ]:
# Load the data

N_min = 2
N_max = 4
lambdas_dict = {}
for n in range(N_min, N_max + 1):
    for p in range(1, n):
        script_dir = os.getcwd()
        concrete_simulations = f"lambdas_N_{n}_p_{p}"
        files_to_find = os.path.join(script_dir, f"data/lamdasOptimized/{concrete_simulations}*")
        list_of_files = glob.glob(files_to_find)
        lambdas_dict[(n, p)] = np.array(load_lambda_means(list_of_files))

# Define the exceptions
exceptions = [(5,4), (6,4), (6,5)]
# Create the figure and axes
fig, axes = plt.subplots(3, 2, figsize=(15, 15))
axes = axes.flatten()

# Colors for different values of p
colors = {1: 'yellow', 2: 'red', 3: 'blue', 4: 'green', 5: 'purple'}
labels_handled = set()

# For each weight we store the maximum value found for all N and p

# Calculate the maximum value for each weight
max_values = np.zeros(5)
for key in lambdas_dict.keys():
    if key not in exceptions:
        max_values = np.maximum(max_values, np.abs(lambdas_dict[key]))

# Plot each weight on a different axis
for weight_index in range(5):
    # Dictionary to store points for each value of p
    points_by_p = {p: ([], []) for p in colors.keys()}

    for key in lambdas_dict.keys():
        if key not in exceptions:
            n, p = key
            weight_value = lambdas_dict[key][weight_index]
            points_by_p[p][0].append(n)
            points_by_p[p][1].append(weight_value)
            label = f'p={p}' if p not in labels_handled else ""
            labels_handled.add(p)

    # Plot the points and lines on the corresponding axis
    ax = axes[weight_index]
    for p, (x, y) in points_by_p.items():
        ax.plot(x, y, 'o-', color=colors[p], label=f'p={p}')
    
    # Add legend and labels
    ax.set_title(f"Weight λ{weight_index + 1} for different N and p")
    ax.set_xlabel("N")
    ax.set_ylabel(f"Value of Weight λ{weight_index + 1}")
    ax.grid(True)

# Add the legend only once
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right')

print("Upper bound distances by factor 0.01: ", calculate_upper_bound_distances(0.01*distances_N_stops_normalized, p))
print("Maximum values of the weights: ", max_values) 
plt.tight_layout()
plt.show()

# As a certain tendency is observed (generalized decreasing in the values or stabilization in certain values), we can set the penalty weights to be the upper bound (max_values) observed and check how accurate is this choice for non computed optimized lamndas.

In [ ]:
# First, an overview fixing N and p

N = 5 # Number of stops
p = 2 # Number of travels, aka number of edges. The number of involucred stops is then p+1
startNode = np.random.randint(0, N-1)
endNode = np.random.randint(startNode+1, N)
num_reads_solver = 400

R = N*(p+1)
distances_N_stops_normalized = distances_original_matrix[:N,:N]/np.max(distances_original_matrix[:N,:N])

lambdas = [max_values[i]-0.5 for i in range(5)] # We try to minmize the weights
Q_matrix,_ = create_QUBO_matrix(distances_N_stops_normalized, p, startNode, endNode, lambdas)

if R <= 20:
    combinations_zipped = brute_force_finding(Q_matrix, distances_N_stops_normalized, p)
    minimal_solution = np.array(list(combinations_zipped[0][0]), dtype=int)

    show_parameters_of_solution(minimal_solution, distances_N_stops_normalized, N, p, startNode, endNode)

    draw_solution_graph(minimal_solution, distances_N_stops_normalized, p, startNode, endNode)

    plot_brute_force_minimums(combinations_zipped, N, p, startNode, endNode, rangePlot=20)

else:
    solution_array, _ = solve_qubo_with_Dwave(Q_matrix, num_reads_solver)

    check_solution(solution_array, N, p, startNode, endNode)

    show_parameters_of_solution(solution_array, distances_N_stops_normalized, N, p, startNode, endNode)

    draw_solution_graph(solution_array, distances_N_stops_normalized, p, startNode, endNode)

    matrix_cost_solution = calculate_cost(solution_array, Q_matrix)

    print("\nTotal Matrix Cost of solution:", matrix_cost_solution)

    random_solution = np.random.randint(0, 2, size=R)

    print("\nTotal Matrix Cost of random solution:", calculate_cost(random_solution, Q_matrix))

In [ ]:
# Now we move systematically through different values of N and p and store for each if a valid solution is found

lambdas = [np.abs(max_values[i]-0.5) for i in range(5)] # We decrease a lot the max weights to see for which N the solutions start ti fail

min_N = 2
max_N = 10
num_reads_solver = 100

count_total_solutions = (max_N * (max_N + 1)) // 2 - ((min_N - 1) * min_N) // 2 - (max_N - min_N + 1)
solutions_analysis = [] 

for n in range(min_N, max_N+1):
    for p in range(1, n):
        startNode = np.random.randint(0, n-1) # Note that each time the code is run, the start and end nodes are randomly selected
        endNode = np.random.randint(startNode+1, n)
        
        distances_N_stops_normalized = distances_original_matrix[:n,:n] / np.max(distances_original_matrix[:n,:n])

        Q_matrix, _ = create_QUBO_matrix(distances_N_stops_normalized, p, startNode, endNode, lambdas)

        solution_array, _ = solve_qubo_with_Dwave(Q_matrix, num_reads_solver)

        is_valid = int(check_solution_return(solution_array, n, p, startNode, endNode))

        # Agregar los datos a la lista
        solutions_analysis.append([n, p, startNode, endNode, solution_array, is_valid])

count_valid_solutions = sum(1 for sol in solutions_analysis if sol[5] == 1)

print(f"Total number of solutions found: {count_total_solutions}")
print(f"Total number of valid solutions found: {count_valid_solutions}")

valid_solutions_per_N = []
no_valid_solutions_per_N = []
for n in range(min_N, max_N+1):
    sum_valid_solutions = sum(1 for sol in solutions_analysis if sol[0] == n and sol[5] == 1)
    sum_all_solutions = sum(1 for sol in solutions_analysis if sol[0] == n)
    valid_solutions_per_N.append(sum_valid_solutions)
    no_valid_solutions_per_N.append(sum_all_solutions - sum_valid_solutions)

valid_solutions_per_p = []
no_valid_solutions_per_p = []
for p in range(1, max_N):
    sum_valid_solutions = sum(1 for sol in solutions_analysis if sol[1] == p and sol[5] == 1)
    sum_all_solutions = sum(1 for sol in solutions_analysis if sol[1] == p)
    valid_solutions_per_p.append(sum_valid_solutions)
    no_valid_solutions_per_p.append(sum_all_solutions - sum_valid_solutions)

# Plot the results in two different axes

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].bar(range(min_N, max_N+1), valid_solutions_per_N, color='green', label='Valid solutions')
axes[0].bar(range(min_N, max_N+1), no_valid_solutions_per_N, bottom=valid_solutions_per_N, color='red', label='No valid solutions')
axes[0].set_xlabel('Number of stops N')
axes[0].set_ylabel('Number of solutions')
axes[0].set_title('Number of valid and no valid solutions found by N')
axes[0].legend()

axes[1].bar(range(1, max_N), valid_solutions_per_p, color='green', label='Valid solutions')
axes[1].bar(range(1, max_N), no_valid_solutions_per_p, bottom=valid_solutions_per_p, color='red', label='No valid solutions')
axes[1].set_xlabel('Number of travels p')
axes[1].set_ylabel('Number of solutions')
axes[1].set_title('Number of valid and no valid solutions found by p')
axes[1].legend()

plt.tight_layout()
plt.show()

# CONCLUSIONS:

We can see that the solutions are more sensitive to the weights when p is close to N, that is why for lower N the solutions start to fail earlier.
This is a promising behaviour since it means that one can compute the optimal lambdas for the first few N, which takes low computational times in the brute force finding and then use these lambdas for higher N, where the solutions are less sensitive to the weights. This is a good strategy to reduce computational times.